In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image

d:\Programs\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def set_device():
    if torch.cuda.is_available():
        dev = "cuda:0"
    else:
        dev = "cpu"
    return torch.device(dev)

In [3]:
device = set_device()

In [4]:
class EnsembleModel(nn.Module):
    def __init__(self, model1, model2, model3, model4):
        super(EnsembleModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
        self.model4 = model4

        # Define additional layers for ensemble if needed

    def forward(self, x):
        output1 = self.model1(x)
        output2 = self.model2(x)
        output3 = self.model3(x)
        output4 = self.model4(x)

        # Perform ensemble strategy (e.g., voting, averaging, etc.)
        ensemble_output = (output1 + output2 + output3 + output4) / 4

        return ensemble_output

In [5]:
# Load the binary classification model
binary_model = torch.load('binary_best_model.pth')
binary_model = binary_model.to(device)
binary_model.eval()

# Load the multi-class classification model
multi_model = torch.load('model.pth')
multi_model = multi_model.to(device)
multi_model.eval()

EnsembleModel(
  (model1): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, trac

In [18]:
# Define the classes for the multi-class classifier
class_names = [
    'Computed Tomography (CT) Scan',
 'Endoscopy',
 'Histology',
 'Magnetic Resonance Imaging (MRI)',
 'Positron Emission Tomography (PET) Scan',
 'Ultrasound',
 'X-ray'
    ]

In [7]:
mean = [0.5817, 0.5569, 0.5599]
std = [0.3154, 0.3083, 0.3114]

# Define the transformation to apply to the images
image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to a specific size
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize(torch.tensor(mean), torch.tensor(std))  # Normalize the image tensors
])

In [11]:
def classify(model, image_transforms, image_path, classes):
    model = model.eval()
    image = Image.open(image_path)
    image = image_transforms(image).float()
    image = image.unsqueeze(0)
    image = image.to(device)

    output = model(image)
    _, predicted = torch.max(output.data, 1)

    print(classes[predicted.item()])

In [12]:
classes = ["Med", "Non-med"]

In [16]:
classify(binary_model, image_transforms, r"D:\SRH\Master Thesis\PMC_testing_data\SDTYAE-000010-034102_1-g006.jpg", classes)

Non-med


In [20]:
classify(multi_model, image_transforms, r"D:\SRH\Master Thesis\PMC_testing_data\PMC1959185_pone.0000867.g001.jpg", class_names)

Magnetic Resonance Imaging (MRI)
